# Dataset Exploration
This is the first notebook in the project. We work with code to do some explorations of the dataset and get insights about the data to use in the project. We use the nibabel library as suggested by Baris Kanber, supervisor of this project.



In [6]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import nibabel as nib

# Working in kaggle, we have all the dataset files in the /input directory ad .nii files (not .nii.gz). 
path_to_ds = "../input/ixi-t1-dataset"
files = glob.glob(f'{path_to_ds}/*.nii')

print("Number of MRIs in the dataset: ", len(files))

In [7]:
# We can load a nii volume like this
vol = nib.load(files[0])

# And we can see how its header looks
h = vol.header
print(h, "\n\n")

# Among other useful info., header shows what simensions we have in the volume
print("Dimensions: ", h['dim'])

# Which matches with:
print('Volume Shape ', vol.shape)

So we have a volume with 3 dimensions, wehere X and Y axis have 256 voxels, while Z axis is limited to 150 voxels. We know now the shape of a single volume, but we are not sure if the whole dataset is homogenous about this. We are going to analyze all the volumes to see if they have similar or identical shapes, which could make the development process easier.








In [10]:
di = {}
dj = {}
dk = {}
for file in files:
    vol = nib.load(file)
    dim = vol.shape
    if vol.shape[0] in di:
        di[vol.shape[0]] += 1
    else:
        di[vol.shape[0]] = 1
    if vol.shape[1] in dj:
        dj[vol.shape[1]] += 1
    else:
        dj[vol.shape[1]] = 1
    if vol.shape[2] in dk:
        dk[vol.shape[2]] += 1
    else:
        dk[vol.shape[2]] = 1
        
print(di)
print(dj)
print(dk)

The x and y axis are fixed, always withe the same amount of voxels, while the z axis is less homogenous, with values in the renge 130..150, being 105 the most common value.

Now, let's see how to get and visualize slices from the volume:

In [23]:
# Based on example in https://nipy.org/nibabel/coordinate_systems.html
# A slice over the third dimension:
z_middle = vol.shape[2] // 2
slice = np.squeeze(vol.slicer[:, :, z_middle-1:z_middle].get_fdata())

# We can then plot the slice as a 2d image
plt.imshow(slice.T, cmap="gray", origin="lower")
plt.show()

This is known as a sagittal view, cause we have cut the volume over the third (Z) dimension. We have got a slice from more or less the middle of the MRI / body's head, where presumably there is more information.

By freezing the other two axis we can obtain the two other typical 2d planes, coronal or frontal and horizontal or tranverse plane:

In [28]:
# An example of coronal view
x_middle = vol.shape[0] // 2
slice = np.squeeze(vol.slicer[x_middle-1:x_middle:,:,:].get_fdata())
plt.imshow(slice, cmap="gray", origin="lower")
plt.show()

# An example of transverse view
y_middle = vol.shape[1] // 2
slice = np.squeeze(vol.slicer[:,y_middle-1:y_middle,:].get_fdata())
plt.imshow(slice, cmap="gray", origin="lower")
plt.show()

Coming back to the NIFTI's header, we find interesting info in the [pixdim attribute](
https://nifti.nimh.nih.gov/nifti-1/documentation/nifti1fields/nifti1fields_pages/pixdim.html/document_view)

In [33]:
print(h['pixdim'])

These values tell us how to pass from voxels to pixels. Some or the values are related to time dimensions, fMRIs, a kind of MRI that we are not interested in

By calling get_zooms(), we get the conversion factor's for each dimension x, y, z or i , j, k:

In [35]:
h.get_zooms()

We see that every voxel represents roughly 1mm3. We notice that the Z dimension has a little less definition (the total amount of surface is represented with less voxels)
